In [ ]:
import torch
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
from langchain.prompts import PromptTemplate

In [ ]:
remote_disk_path = '/home/kikarpenko_1/data'

In [ ]:
# Загрузка модели
model = AutoModelForCausalLM.from_pretrained(remote_disk_path)

# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained(remote_disk_path)

In [1]:
def get_embedding(sentence, tokenizer, model):
    # Tokenize sentences
    inputs = tokenizer.encode_plus(sentence, return_tensors="pt", padding=True, truncation=True)

    # Compute token embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract hidden states from model output
    hidden_states = outputs.hidden_states

    if hidden_states is not None and len(hidden_states) > 0:
        # Take the last layer hidden state
        last_hidden_state = hidden_states[-1]

        # Mean Pooling - Take attention mask into account for correct averaging
        attention_mask = inputs["attention_mask"]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        sentence_embeddings = sum_embeddings / sum_mask

        # Normalize embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

        return sentence_embeddings
    else:
        raise ValueError("No hidden states found in model output.")


In [ ]:
my_info_prompt = ("<s>system \nТы — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.</s> \n<s>user \nТекст: {prompt} \nВопрос: {question}</s> \n<s>bot \nВот ответ на ваш вопрос длиной не более 50 слов:")

In [1]:
def get_answer(info_prompt, question, prompt):

    prompt = info_prompt.format(question=question, prompt=prompt)
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(input_ids=inputs["input_ids"],
                            top_p=0.5,
                            temperature=0.3,
                            attention_mask=inputs["attention_mask"],
                            max_new_tokens=100,
                            pad_token_id=tokenizer.eos_token_id,
                            do_sample=True)

    output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    parsed_answer = output.split("Вот ответ на ваш вопрос длиной не более 50 слов:")[1].strip()

    if "bot:" in parsed_answer:
        parsed_answer = parsed_answer.split("bot:")[0].strip()

    # parsed_answer = output.split("bot:")[1].strip()
    return parsed_answer

In [2]:
questions_hse = [
    "Какие факультеты и программы предлагает НИУ ВШЭ?",
    "Каковы требования для поступления в НИУ ВШЭ?",
    "Какие документы необходимо подать для поступления в НИУ ВШЭ?",
    "Есть ли вступительные экзамены или тесты для поступления в НИУ ВШЭ, и как к ним подготовиться?",
    "Какова стоимость обучения на различных программах в НИУ ВШЭ?",
    "Предоставляются ли стипендии или гранты для студентов НИУ ВШЭ?",
    "Какие карьерные возможности открываются для выпускников НИУ ВШЭ?",
    "Есть ли у НИУ ВШЭ международные программы обмена студентами?",
    "Какие возможности для стажировок и практики доступны студентам НИУ ВШЭ?",
    "Какова средняя группа студентов на лекциях и семинарах в НИУ ВШЭ?",
    "Есть ли общежития для иногородних студентов в НИУ ВШЭ?",
    "Как организована внеучебная деятельность и студенческая жизнь в НИУ ВШЭ?",
    "Есть ли в НИУ ВШЭ спортивные секции и клубы по интересам?",
    "Какие возможности для научной работы и исследований предоставляются студентам НИУ ВШЭ?",
    "Какова процедура подачи заявки для иностранных студентов в НИУ ВШЭ?",
    "Какой уровень владения языком требуется для поступления в НИУ ВШЭ?",
    "Какие услуги предоставляет НИУ ВШЭ для поддержки студентов (например, психологическая помощь, карьерные консультации)?",
    "Каковы сроки подачи документов и зачисления в НИУ ВШЭ?",
    "Есть ли у НИУ ВШЭ партнерские отношения с крупными компаниями или предприятиями?",
    "Какова процедура апелляции в случае несогласия с результатами вступительных экзаменов в НИУ ВШЭ?"
]

In [ ]:
def get_cos_sim(question):
    cos_sim = F.cosine_similarity(emb_database, emb, dim=1, eps=1e-8)
    return cos_sim

In [ ]:
answers = []
emb_database = torch.empty((0, 384), dtype=torch.float32)

In [4]:
import pandas as pd
csv_file_path = 'context_data.csv'

# Чтение данных из CSV файла в DataFrame
data = pd.read_csv(csv_file_path)

# Вывод первых нескольких строк DataFrame
print(data)

                                  Processed_Sentences
0   {'title': 'Абитуриентам бакалавриата НИУ ВШЭ—П...
1   ', 'Образование', 'Стажер посещает занятия на\...
2   ', 'Эксперты НИУ ВШЭ – Пермь приняли участие в...
3   ', 'Подать заявку\xa0Задать вопрос\xa0Записать...
4   {'title': 'О Вышке в Нижнем Новгороде – Национ...
5   ', 'Требования к кандидатам:', '—\xa0наличие с...
6   {'title': 'Образовательный онлайн-проект «Карь...
7   ', 'Задать вопрос о программе Поступить', 'О п...
8   ', 'Образование', 'Стажер посещает занятия на\...
9   {'title': 'Аналитический центр – Национальный ...
10  {'title': 'Тема «общежития НИУ ВШЭ» – Новости ...
11  ', 'Основное образование', 'Бакалавриат', '16'...
12  {'title': 'Что в меню и есть ли очереди – Ново...
13  {'title': 'Факультет экономических наук – Наци...
14  ', 'Особенности организации перевода студентов...
15  {'title': 'Вход', 'description': '', 'url': 'h...
16  {'title': 'Медицинская и психологическая помощ...
17  {'title': 'Учебная лабор

In [5]:
#генерация ответа до 50 токенов
for i in range(len(questions_hse)):
  context = data[i]
  emb = get_embedding(questions_hse[i])

  cos_sim = get_cos_sim(questions_hse[i])
  answer = get_answer(my_info_prompt, questions_hse[i], context)
  emb_database = torch.cat((emb_database, emb), 0)
  answers.append(answer)
  get_cos_sim(questions_hse[i])
  print(questions_hse[i])
  print(f'Answer from model: {answer} \n')

KeyError: 0